In [1]:
import sys
sys.path.append('../../')

In [68]:
from src.preprocessor.preprocessing import FileIO
from src.pipelines.pipeline import create_dataset, create_vectors
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import torch
import numpy as np

In [4]:
data_path = '../../data/huberman_labs.json'

In [5]:
data = FileIO().load_json(data_path)

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/anaconda/envs/openai/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [38]:
summaries = [d['summary'] for d in data]
sum_matrix = model.encode(summaries, convert_to_tensor=True)

In [39]:
for i, d in enumerate(data):
    d['summary_embedding'] = sum_matrix[i]

### Inference

In [142]:
topic = 'Testosterone replacement therapy'
topic_vector = model.encode(topic, convert_to_tensor=True).view(1,-1)

In [143]:
answer_tensor = cos_sim(sum_matrix, topic_vector).cpu()

In [144]:
sorted_answer_tensor = answer_tensor.squeeze().argsort(descending=True)

In [145]:
def grab_titles(indexes: list[int], data: list[dict]) -> list[str]:
    indexes = indexes.tolist()
    return [data[i]['title'] for i in indexes]
    

In [146]:
grab_titles(sorted_answer_tensor[:7], data)

['Dr. Kyle Gillett: Tools for Hormone Optimization in Males | Huberman Lab Podcast 102',
 'Dr. Michael Eisenberg: Improving Male Sexual Health, Function & Fertility',
 'Dr. Duncan French: How to Exercise for Strength Gains & Hormone Optimization | Huberman Lab #45',
 'Dr. Kyle Gillett: How to Optimize Your Hormones for Health & Vitality | Huberman Lab Podcast #67',
 'The Science of How to Optimize Testosterone & Estrogen',
 'Biological Influences On Sex, Sex Differences & Preferences',
 'Understanding & Controlling Aggression | Huberman Lab Podcast #71']

In [147]:
answer_tensor.squeeze().sort(descending=True)

torch.return_types.sort(
values=tensor([ 0.5858,  0.4801,  0.4113,  0.4095,  0.4090,  0.3695,  0.3602,  0.3415,
         0.3174,  0.3154,  0.3100,  0.2865,  0.2819,  0.2809,  0.2752,  0.2750,
         0.2735,  0.2693,  0.2659,  0.2547,  0.2542,  0.2515,  0.2487,  0.2413,
         0.2412,  0.2408,  0.2393,  0.2390,  0.2384,  0.2359,  0.2355,  0.2305,
         0.2281,  0.2279,  0.2257,  0.2252,  0.2224,  0.2191,  0.2156,  0.2124,
         0.2118,  0.2109,  0.2097,  0.2091,  0.2066,  0.2060,  0.2059,  0.2043,
         0.2011,  0.2009,  0.1993,  0.1983,  0.1968,  0.1954,  0.1901,  0.1900,
         0.1861,  0.1851,  0.1838,  0.1831,  0.1815,  0.1804,  0.1791,  0.1740,
         0.1739,  0.1727,  0.1717,  0.1703,  0.1698,  0.1679,  0.1634,  0.1597,
         0.1594,  0.1579,  0.1574,  0.1559,  0.1545,  0.1544,  0.1540,  0.1532,
         0.1499,  0.1472,  0.1447,  0.1432,  0.1420,  0.1419,  0.1419,  0.1412,
         0.1408,  0.1379,  0.1375,  0.1360,  0.1353,  0.1301,  0.1286,  0.1285,
        

In [157]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
  """Returns the number of tokens used by a list of messages."""
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

In [167]:
messages = [
  {"role": "system", "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English."},
  {"role": "system", "name":"example_user", "content": "New synergies will help drive top-line growth."},
  {"role": "system", "name": "example_assistant", "content": "Things working well together will increase revenue."},
  {"role": "system", "name":"example_user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
  {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
]

model = "gpt-3.5-turbo-0613"

print(f"{num_tokens_from_messages(messages, model)} prompt tokens counted.")
# Should show ~126 total_tokens

104 prompt tokens counted.


In [159]:
from src.llm.llm_utils import load_azure_openai

In [160]:
llm = load_azure_openai()

In [162]:
from openai import OpenAI
client = OpenAI()

In [168]:
response = client.chat.completions.create(model='gpt-3.5-turbo-0125', messages=messages, temperature=0)

In [169]:
response.usage.total_tokens

131

In [170]:
response.usage.prompt_tokens

106